# Python Generators

## Generator Functions & Expressions

### Build a Generator Function

There is a lot of overhead in building an iterator in Python. We have to implement a class with `__iter__()` and `__next__()` method, keep track of internal states, raise StopIteration when there was no values to be returned etc.

This is both lengthy and counter intuitive. Generator comes into rescue in such situations.

Python generators are a simple way of creating iterators. All the overhead we mentioned above are automatically handled by generators in Python.

Simply speaking, a generator is a function that returns an object (iterator) which we can iterate over (one value at a time).

In [1]:
# function solution
def even_integers_function(n):
    result = []
    for i in range(n):
        if i % 2 == 0:
            result.append(i)
    return result

In [2]:
# generator solution
def even_integers_generator(n):
    for i in range(n):
        if i % 2 == 0:
            yield i

In [3]:
even_integers_function(10)

[0, 2, 4, 6, 8]

In [4]:
gen = even_integers_generator(10)
gen

<generator object even_integers_generator at 0x0000000F8F04CE58>

In [5]:
print(next(gen))
print(next(gen))
print(next(gen))
print(next(gen))
print(next(gen))

0
2
4
6
8


### Use a Generator Expression

There is also a short way to build generators like list comprehensions. These are called generator expressions. 

* List comprehension  
`newlist = [item.upper() for item in collection]`  
  

* generator expression  
`(item.upper() for item in collection)`  


In [6]:
#list of mixed format numbers
numbers = [7, 22, 4.5, 99.7, '3', '5']

#convert numbers to integers using expression
integers = (int(n) for n in numbers)
integers

<generator object <genexpr> at 0x0000000F8F3F6570>

In [7]:
list(integers)

[7, 22, 4, 99, 3, 5]

In [8]:
names_list = ['Adam','Anne','Barry','Brianne','Charlie','Cassandra','David','Dana']

#Converts names to uppercase
uppercase_names = (name.upper() for name in names_list)
uppercase_names

<generator object <genexpr> at 0x0000000F8F3F66D8>

In [9]:
list(uppercase_names)

['ADAM', 'ANNE', 'BARRY', 'BRIANNE', 'CHARLIE', 'CASSANDRA', 'DAVID', 'DANA']

In [10]:
even_integers = (n for n in range(10) if n % 2 == 0)
even_integers

<generator object <genexpr> at 0x0000000F8F3F68B8>

In [11]:
list(even_integers)

[0, 2, 4, 6, 8]

In [13]:
# list of names
names_list = ['Adam','Anne','Barry','Brianne','Charlie','Cassandra','David','Dana']

# too long
# reverse_uppercase = (name[::-1] for name in (name.upper() for name in names_list))

# breaking it up 
uppercase = (name.upper() for name in names_list)
reverse_uppercase = (name[::-1] for name in uppercase)

In [14]:
list(reverse_uppercase)

['MADA', 'ENNA', 'YRRAB', 'ENNAIRB', 'EILRAHC', 'ARDNASSAC', 'DIVAD', 'ANAD']

### Fibonacci Sequence

Let's take a look at the [Fibonacci Sequence](https://www.mathsisfun.com/numbers/fibonacci-sequence.html) with generators.

In [15]:
# Fibonacci Sequence Generator
def fibonacci_gen():
    trailing, lead = 0,1
    while True:
        yield lead
        trailing, lead = lead, trailing+lead

In [16]:
fib = fibonacci_gen()

In [17]:
for _ in range(10):
    print(next(fib))

1
1
2
3
5
8
13
21
34
55


### Building Generator Pipeline

Generators can also be chained together to build a generator pipeline. Using the `names.txt` file, we will pipeline the generator process by stripping off the whitespace and retrieving the full names, followed by zipping name and their lengths and lastly finding the longest name from our names. Without generators, we would have to do these steps one by one.

In [19]:
# using generators to find the longest name
full_names = (name.strip() for name in open('names.txt'))
lengths = ((name, len(name)) for name in full_names)
longest = max(lengths, key=lambda x: x[1])

In [20]:
longest

('Brittanie Talamantes', 20)

In [18]:
# adding separate_names generator as another stage in pipeline
def separate_names(names):
    for full_name in names:
        for name in full_name.split(' '):
            yield name

In [21]:
full_names = (name.strip() for name in open('names.txt'))
names = separate_names(full_names)
lengths = ((name, len(name)) for name in names)
longest = max(lengths, key=lambda x:x[1])

In [22]:
longest

('Humbertson', 10)

In [23]:
def get_longest(namelist):
    full_names = (name.strip() for name in open(namelist))
    names = separate_names(full_names)
    lengths = ((name, len(name)) for name in names)
    return max(lengths, key=lambda x:x[1])

In [24]:
get_longest('names.txt')

('Humbertson', 10)

## Using Generators As Context Managers

### Context Manager Overview

In real systems, it's difficult to make sure that close() is called on every file opened, especially if the file is in a function that may raise an exception or has multiple return paths. In a complicated function that opens a file, how can you possibly be expected to remember to add close() to every place that function could return from? And that's not counting exceptions, either (which may happen from anywhere). The short answer is: you can't be.

In other languages, developers are forced to use `try...except...finally` every time they work with a file (or any other type of resource that needs to be closed, like sockets or database connections). Python gives us a simple way to make sure all resources we use are properly cleaned up, regardless of if the code returns or an exception is thrown: context managers.

```
@contextmanager
def simple_context_manager(obj):
    try:
        #do something
        yield
    finally:
        #wrap up
```

### Context Manager Decorator

In [26]:
# increments some_property by 1
from contextlib import contextmanager

@contextmanager
def simple_context_manager(obj):
    try:
        obj.some_property += 1
        yield
    finally:
        obj.some_property -= 1
        
class Simple_obj(object):
    def __init__(self, arg):
        self.some_property = arg

In [27]:
s = Simple_obj(5)
with simple_context_manager(s):
    print(s.some_property)

6


Using with, we can call anything that returns a context manager (like the built-in open() function). We assign it to a variable using `... as <variable_name>`. When the variable goes out of scope, it automatically calls a special method that contains the code to clean up the resource.

In [28]:
s.some_property

5

But where is the code that is actually being called when the variable goes out of scope? The answer is, "wherever the context manager is defined." There are a number of ways to create a context manager.  

The simplest is to define a class that contains two special methods:  

* `__enter__()` returns the resource to be managed (like a file object in the case of open()).
* `__exit__()` does any cleanup work and returns nothing.

In [29]:
from contextlib import contextmanager
from logging import Logger, FileHandler

In [30]:
# How it is done usually
class Other_scm():
    def __init__(self, obj):
        self.obj = obj
    def __enter__(self):
        self.obj.some_property+=1
    def __exit__(self, *args):
        self.obj.some_property-=1

In [31]:
# a more complex example
@contextmanager
def error_logging(logger, level):
    oldlevel = logger.level
    try:
        logger.setLevel(level)
        yield
    finally:
        logger.setLevel(oldlevel)

In [32]:
logger = Logger('name',20)
handler = FileHandler('flog.log')
logger.addHandler(handler)
logger.info('this will get logged')

In [33]:
with error_logging(logger, 30):
    logger.info('this will not get logged')
logger.info('this will get logged because the level is {}'.format(logger.level))

### Building HTML File with Context Manager

In [35]:
from time import time

HEADER = "this is the header \n"
FOOTER = "\nthis is the footer \n"


@contextmanager
def new_log_file(name):    
    try:
        logname = name
        f = open(logname, 'w')
        f.write(HEADER)
        yield f
    finally:
        f.write(FOOTER)
        print("logfile created")
        f.close()

In [37]:
with new_log_file('new_log.log') as lg:
    lg.write('This is something in the middle from header and footer')

logfile created


## Coroutines

### Coroutines Overview

<span class="mark">TODO: ADD EXPLANATIONS ABOUT THE TOPIC.</span>

In [ ]:
def coroutine_example():
    while True:
        x = yield
        #do something with x
        print x

### Create Coroutines

<span class="mark">TODO: ADD EXPLANATIONS ABOUT THE TOPIC.</span>

In [ ]:
def counter(string):
    count = 0
    try:
        while True:
            item = yield
            if isinstance(item, str):
                if item in string:
                    count += 1
                    print item
                else:
                    print 'No Match'
            else:
                print 'Not a string'
    except GeneratorExit:
        print count

### Build a Coroutine Decorator

<span class="mark">TODO: ADD EXPLANATIONS ABOUT THE TOPIC.</span>

In [ ]:
def coroutine_decorator(func):
    def wrap(*args, **kwargs):
        cr = func(*args, **kwargs)
        cr.next()
        return cr
    return wrap


@coroutine_decorator
def coroutine_example():
    while True:
        x = yield
        #do something with x
        print x

### Consume Values with Send Method

<span class="mark">TODO: ADD EXPLANATIONS ABOUT THE TOPIC.</span>

In [ ]:
from coroutine_decorator import coroutine_decorator

def sender(filename, target):
    for line in open(filename):
        target.send(line)
    target.close()
        

@coroutine_decorator
def match_counter(string):
    count = 0
    try:
        while True:
            line = yield
            if string in line:
                count += 1
    except GeneratorExit:
        print '{}: {}'.format(string, count)



@coroutine_decorator
def longer_than(n):
    count = 0
    try:
        while True:
            line = yield
            if len(line)>n:
                print line
                count += 1
    except GeneratorExit:
        print 'longer than {}: {}'.format(n, count)

### Coroutine Pipelines

<span class="mark">TODO: ADD EXPLANATIONS ABOUT THE TOPIC.</span>

In [ ]:
from coroutine_decorator import coroutine_decorator

@coroutine_decorator
def router():
    try:
        while True:
            line = yield
            (first, last) = line.split(' ')
            fnames.send(first)
            lnames.send(last.strip())
    except GeneratorExit:
        fnames.close()
        lnames.close()
        
        

@coroutine_decorator
def file_write(filename):
    try:
        with open(filename,'a') as file:
            while True:
                line = yield
                file.write(line+'\n')
    except GeneratorExit:
        file.close()
        print 'one file created'

if __name__ == "__main__":
    fnames = file_write('first_names.txt')
    lnames = file_write('last_names.txt')
    router = router()
    for name in open('names.txt'):
        router.send(name)
    router.close()